In [1]:
# Python 2
import pickle
with open('df.pickle', mode='rb') as h:
    df = pickle.load(h)
with open('hex.pickle', mode='rb') as h:
    label_count = len(pickle.load(h)['id_name'])

In [2]:
import cv2
import numpy as np
from os.path import join
img_dir = '../pascal12/JPEGImages'
img_mean = np.load('../ilsvrc_2012_mean.npy').astype(np.float32)  # 3*256*256
img_mean = np.swapaxes(np.swapaxes(img_mean, 0, 1), 1, 2)  # convert to XY[BGR]
def load_image(filename):
    """
    Loads image, subtract mean, resize, change axis order from XY[BGR] to [BGR]XY
    Returns:
        Result image as an np.float32 array with size 3*227*227. Values within [0, 255]
    """
    img = cv2.resize(cv2.imread(join(img_dir, filename)), (256, 256))
    img = cv2.resize(img.astype(np.float32) - img_mean, (227, 227))
    return np.swapaxes(np.swapaxes(img, 1, 2), 0, 1)

In [3]:
import h5py
for split in ['train', 'test']:
    df_split = df[df['split']==split]
    X = map(load_image, df_split['img'])
    Y = np.zeros((len(X), label_count), dtype=np.float32)  # labels in np.float32 for error calculation
    pseudo_labels = df_split['pseudo_label'].reset_index(drop=True)
    for i in range(0, len(X)):
        Y[i, pseudo_labels[i]] = 1
    with h5py.File('../{}.h5'.format(split), 'w') as h:
        h.create_dataset('data', data=np.array(X, dtype=np.float32))
        h.create_dataset('label', data=Y)